In [ ]:
%pip install wandb
%pip install ultralytics
%pip install matplotlib
%pip install numpy
%pip install opencv-python
%pip install inference

In [ ]:
ZERO: int = 0
ONE: int = 1

ITERATION: int  = 1
IOU: int        = 80
EPOCH: int      = 6
BATCHES: int    = 9

In [ ]:
def calculateIOU() -> float:
    global IOU

    return float(
        IOU
        /
        100
    )

In [ ]:
from ultralytics    \
    import YOLO

from math           \
    import (
        floor
)

from random         \
    import (
        SystemRandom
)

from os.path        \
    import (
        join
)

from os \
    import environ

from logging import ( 
    getLogger, 
    ERROR
)

from wandb import (
    init, 
    finish
)

from wandb.integration.ultralytics \
    import (
        add_wandb_callback
)

In [ ]:
environ['WANDB_DIR'] = '~/wandb'

environ['WANDB_CONSOLE']       = 'off'
environ['WANDB_DISABLE_CODE']  = 'off'
environ['WANDB_NOTEBOOK_NAME'] = 'TrainingOfModel.ipynb'
environ['WANDB_ENTITY']        = 'Designermadsen'
environ['WANDB_SILENT']        = 'True'


environ['YOLO_MODEL_ITERATION'] = str(
    ITERATION
)

environ['YOLO_WEIGHT_MODEL_VERSION'] = \
    str(
        '/mnt/c/Users/Kentv/Desktop/Spectre/runs/detect/train6/weights/best.pt'
)

environ['YOLO_MODEL_VERSION']        = \
    'yolo11m.pt'



environ['TRAINING_EPOCH'] = str(
    EPOCH
)

environ['TRAINING_BATCHES'] = str(
    BATCHES
)

environ['TRAINING_PATIENCE'] = str(
    0
)

environ['TRAINING_WORKERS'] = str(
    20
)

In [ ]:
def get_weight_location() -> str:
    global MODEL_WEIGHT_LOCATION
    return MODEL_WEIGHT_LOCATION

def set_weight_location(
    value: str
) -> None:
    global MODEL_WEIGHT_LOCATION
    MODEL_WEIGHT_LOCATION = value

def get_current_model_iteration() -> int:
    global MODEL_ITERATION_VERSION
    return MODEL_ITERATION_VERSION

def set_current_model_iteration(
    value: int
) -> None:
    global MODEL_ITERATION_VERSION
    MODEL_ITERATION_VERSION = value

def increment_of_model() -> None:
    global ONE

    set_current_model_iteration(
        get_current_model_iteration() + ONE
    )

def generate_weight_location() -> str:
    weight_path = join(
        str(
            './runs/detect/train' 
        )        
        + 
        str(
            get_current_model_iteration()
        ),
        'weights/best.pt'
    )

    set_weight_location(
        weight_path
    )

    return get_weight_location()

def getSeed() -> int:
    global SEED
    
    if isSeedNone():
        return generateSeed()

    return SEED

def setSeed(
    value: int
) -> None:
    global SEED
    SEED = value


def generateGenerator() -> None:
    global generator

    if generator is None:
        generator = SystemRandom()

def isSeedNone() -> bool:
    global SEED
    return SEED is None

def maximumSizeForInteger() -> int:
    return 2147483647

def generateSeed() -> None:
    global          \
        SEED,       \
        generator,  \
        ZERO
    
    generateGenerator()

    setSeed(
        generator.randint(
            ZERO, 
            maximumSizeForInteger()
        )
    )

    return SEED

def getModel():
    global model
    return model

def setModel(
    value
):
    global model
    model = value

def modelLoadWeights() -> None:
    global MODEL_WEIGHT_LOCATION

    if not (
        MODEL_WEIGHT_LOCATION is None
    ):
        getModel().load(
            get_weight_location()
        )

def initializeLogging() -> None:
    logger = getLogger(
        __name__
    )

    logger.setLevel(
        ERROR
    )

In [ ]:
generator   = None
model       = None
logger      = None
model       = None

NUMBER_OF_TIMES_TO_ITERATE_TRAINING: int = ONE

MODEL_ITERATION_VERSION: int = int(
    environ['YOLO_MODEL_ITERATION']
)

FIRST_MODEL_TO_LOAD: str = environ['YOLO_WEIGHT_MODEL_VERSION']

FINISH_WHEN_DONE: bool = True

EPOCHS: int     = int(
    environ['TRAINING_EPOCH']
)

BATCH: int      = int(
    environ['TRAINING_BATCHES']
)

PATIENCE: int   = int(
    environ['TRAINING_PATIENCE']
)


IMAGE_WIDTH_SIZE: int = 640
IMAGE_HEIGHT_SIZE: int = 640

IMAGE_DIMENSIONS: tuple = (
    IMAGE_HEIGHT_SIZE, 
    IMAGE_WIDTH_SIZE
)

SEED = None

IS_TO_RESUME: bool = False

#
MOSAIC_END_AT: int = -ONE

PROJECT_NAME: str = 'spectre'

HOME_LOCATION: str = '/home/ai'

DATASET_LOCATION: str = join(
    HOME_LOCATION,
    'dataset.yaml'
)

MODEL_WEIGHT_LOCATION: str | None = None

#
DEVICES_TO_USE: list = [
    ZERO
]

In [ ]:
def getTrainingWorkers() -> int:
    return int(
        environ['TRAINING_WORKERS']
    )

In [ ]:
initializeLogging()

init(
    project     = PROJECT_NAME, 
    job_type    = 'training',
    save_code   = False
)

setModel(
    YOLO(
        FIRST_MODEL_TO_LOAD,
        verbose = False
    )
)

add_wandb_callback(
    model,
    enable_train_validation_logging     = False,
    enable_validation_logging           = False,
    enable_prediction_logging           = False,
    enable_model_checkpointing          = False
)


In [ ]:
if not IS_TO_RESUME:
    generateSeed()

for idx in range(
    NUMBER_OF_TIMES_TO_ITERATE_TRAINING
):
    training_results =                          \
        model.train(
            data         = DATASET_LOCATION, 

            batch        = int(
                environ['TRAINING_BATCHES']
            ), 
            epochs       = int(
                environ['TRAINING_EPOCH']
            ),
            cos_lr       = True,
            box          = 3.0,
            iou          = calculateIOU(),
            dropout      = 0.075,
            patience     = PATIENCE,
            
            lrf          = 0.01,
            conf         = 0.001,
            weight_decay = 0.001,

            auto_augment = 'randaugment',
            bgr          = 0.05,
            hsv_h        = 0.15,
            hsv_s        = 0.15,
            hsv_v        = 0.15,

            translate    = 0.05,
            scale        = 0.05,
            mixup        = 0.40,
            momentum     = 0.95,
            mosaic       = 0.40,
            
            close_mosaic  = MOSAIC_END_AT,
            
            imgsz         = IMAGE_DIMENSIONS,
            multi_scale   = False,
            resume        = IS_TO_RESUME,
            seed          = getSeed(),
            workers       = getTrainingWorkers(),
            stream_buffer = True,
            device        = DEVICES_TO_USE,
            
            save          = True,

            plots         = True,
            show          = False,
            verbose       = False
    )
    
    if NUMBER_OF_TIMES_TO_ITERATE_TRAINING > ONE:
        increment_of_model()

    if not (
        int( 
            NUMBER_OF_TIMES_TO_ITERATE_TRAINING 
            - 
            ONE
        )
        == 
        idx
    ):
        modelLoadWeights()


In [ ]:
if FINISH_WHEN_DONE:
    finish()